In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy Release
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query1 = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""
avg_price_4_bedrooms = spark.sql(query1)
avg_price_4_bedrooms.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query2 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""
avg_price_3_bed_3_bath = spark.sql(query2)
avg_price_3_bed_3_bath.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query3 = """
SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built
"""
avg_price_3_bed_3_bath_2_floor_2000sqft = spark.sql(query3)
avg_price_3_bed_3_bath_2_floor_2000sqft.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query4 = """
SELECT view, ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING average_price >= 350000
ORDER BY average_price DESC
"""
avg_price_per_view = spark.sql(query4)
avg_price_per_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 1.6733367443084717 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_view_cached = spark.sql(query4)
avg_price_per_view_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 3.3814380168914795 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_partitioned")

In [15]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_partitioned")

In [16]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_per_view_parquet = spark.sql(query4)
avg_price_per_view_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 0.485959529876709 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [19]:
# 15. Check if the home_sales is no longer cached
print(spark.catalog.isCached("home_sales"))


False
